In [1]:
import optuna

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [7]:
data_dir = "../data/Sulfur_Models/"
model_type = 'NN'
target = 'Tavg'

In [9]:
def objective(trial):   
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 1, 10000, log=True)
        epochs = trial.suggest_int("epochs", 1, 20)
        rmse = build_train_test_model(data_dir, model_type, target, scale, n_layers, n_hidden_units, batch_size, epochs, trial)
    elif model_type == 'XGBoost':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 50, 10000, log=True)
        rmse = build_train_test_model(data_dir, model_type, target, scale, n_estimators)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 100, log=True)
        rmse = build_train_test_model(data_dir, model_type, target, scale, n_estimators)
    
    return rmse

In [10]:
study = optuna.create_study(direction='minimize')

[I 2022-06-29 12:52:23,295] A new study created in memory with name: no-name-06aa1eed-a859-4b35-8b99-002209b8faad


In [ ]:
study.optimize(objective, n_trials=5)

2022-06-29 12:52:38.717811: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-29 12:52:38.797398: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/10
 22585/505272 [>.............................] - ETA: 4:42 - loss: 0.0085